# Author Topic Model 평가하기

* hellinger distance : 사용자간의 유사성(토픽의 확률분포)을 평가하기 위한 measure.

In [1]:
from gensim.models import AuthorTopicModel
from gensim.corpora import Dictionary, bleicorpus
from gensim import corpora

from gensim.matutils import hellinger
from gensim import matutils

from pprint import pprint
import pandas as pd
import os

In [2]:
model = AuthorTopicModel.load('data/kakao_ATM_model')  # NUM_TOPICS = 4

In [3]:
# 토픽별 분포 확인하기
model.show_topic(0)

[('나', 0.010368158594930087),
 ('나도', 0.005641677219910483),
 ('다', 0.00549259642870929),
 ('완전', 0.004452450579120248),
 ('그래서', 0.003992346335516386),
 ('내가', 0.003966104395676567),
 ('오', 0.0037888147726983542),
 ('역시', 0.0036775522938288637),
 ('근데', 0.003638984283893234),
 ('어제', 0.00346218867638447)]

In [4]:
# 토픽 라벨 지정.
topic_labels = ["Topic0", "Topic1", "Topic2", "Topic3"]

In [5]:
# 토픽별로 topN 단어 확인하기.

for topic in model.show_topics(model.num_topics):
    print('Label: ' + topic_labels[topic[0]])
    words = ''
    for word, prob in model.show_topic(topic[0], topn=20):
        words += word + ' '
    print('Words: ' + words)
    print()

Label: Topic0
Words: 나 나도 다 완전 그래서 내가 오 역시 근데 어제 진짜 오늘 난 다들 이 있어 그 이거 그럼 너무 

Label: Topic1
Words: 내가 나 형 부끄 누나 지금 근데 없어 나도 헐 더 좀 난 다 그럼 형산군 넌 저거 유희야 단발이 

Label: Topic2
Words: 뭐야 우리 이렇게 내일 아마 이미 다 잘 음 다들 나도세자 낄끼빠빠 어디 지금 오빠가 진짜 어떻게 같이 맞어 그럼 

Label: Topic3
Words: 아 왜 우리 난 그래 응 근데 아니야 엄청 그건 또 내가 나도 오빠 나왕왕 다시 다 더 진짜 아니 



In [6]:
# 토픽별 사용자의 분포 확인

model['WH']

[(0, 0.6516586242497926),
 (1, 0.02781274148041911),
 (2, 0.026368042747998623),
 (3, 0.29416059152178975)]

In [7]:
def show_author(name):
    print(name)
    print('Docs:', len(model.author2doc[name]))
    print('Topic distribution:')
    pprint([(topic_labels[topic[0]], topic[1]) for topic in model[name]])
    print()

In [8]:
for author in model.id2author.values():
    show_author(author)

BJ
Docs: 4222
Topic distribution:
[('Topic0', 0.772028304483581),
 ('Topic1', 0.2032355385592529),
 ('Topic3', 0.024471814088761153)]

BR
Docs: 2824
Topic distribution:
[('Topic0', 0.01944961099706222),
 ('Topic1', 0.047728566224437004),
 ('Topic2', 0.16202851103825855),
 ('Topic3', 0.7707933117402422)]

GM
Docs: 554
Topic distribution:
[('Topic1', 0.015272984401152146),
 ('Topic2', 0.06236817178775477),
 ('Topic3', 0.9131162150121103)]

HL
Docs: 3554
Topic distribution:
[('Topic0', 0.38160590843995945),
 ('Topic1', 0.0842289410448531),
 ('Topic2', 0.01536420351728692),
 ('Topic3', 0.5188009469979006)]

KS
Docs: 2145
Topic distribution:
[('Topic0', 0.5258486573403207),
 ('Topic1', 0.020486557930329042),
 ('Topic2', 0.29929543270365705),
 ('Topic3', 0.1543693520256932)]

WH
Docs: 504
Topic distribution:
[('Topic0', 0.6516586242497926),
 ('Topic1', 0.02781274148041911),
 ('Topic2', 0.026368042747998623),
 ('Topic3', 0.29416059152178975)]

YH
Docs: 1108
Topic distribution:
[('Topic0', 0.0

In [9]:
# Hellinger Distance를 이용하여 비슷한 토픽을 가진 사용자를 추정하는 함수.

# author-topic 분포 만들기
author_vecs = [model.get_author_topics(author) for author in model.id2author.values()]

def similarity(vec1, vec2):
    # vec1, vec2 사이의 hellinger similarity 구하기.
    dist = hellinger(matutils.sparse2full(vec1, model.num_topics), \
                              matutils.sparse2full(vec2, model.num_topics))
    sim = 1.0 / (1.0 + dist)
    return sim
 
def get_sims(vec):
    # 각 사용자들 사이의 similarity pair 구하기.
    sims = [similarity(vec, vec2) for vec2 in author_vecs]
    return sims
 
def get_table(name, top_n=10, smallest_author=1):
    """
    주어진 사용자에 대해서 topN 사람만큼 유사도를 정렬해서 table을 출력하는 함수.
    """
    
    # 유사도 측정하기
    sims = get_sims(model.get_author_topics(name))
 
    # 저자별 정보 정렬하기
    table = []
    for elem in enumerate(sims):
        author_name = model.id2author[elem[0]]
        sim = elem[1]
        author_size = len(model.author2doc[author_name])
        if author_size >= smallest_author:
            # 문서가 너무 적은 사용자는 제외
            table.append((author_name, sim, author_size))
            
    # 사용자 패턴 분석 결과를 Dataframe으로 만들기
    df = pd.DataFrame(table, columns=['Author', 'Score', 'Size'])
    df = df.sort_values('Score', ascending=False)[:top_n]
    
    return df

In [10]:
# 사용자별 대화 패턴 검증
get_table('WH')

,Author,Score,Size
5,WH,1.000000,504
3,HL,0.829679,3554
4,KS,0.770405,2145
0,BJ,0.734586,4222
6,YH,0.678453,1108
1,BR,0.642556,2824
2,GM,0.607925,554


In [11]:
get_table('KS')

,Author,Score,Size
4,KS,1.000000,2145
5,WH,0.770405,504
3,HL,0.714481,3554
0,BJ,0.672703,4222
1,BR,0.645184,2824
6,YH,0.630066,1108
2,GM,0.594401,554
